# 1. Zero-Shot Prompting - Prompts Sin Ejemplos Previos

## Objetivos de Aprendizaje
- Comprender los principios del zero-shot prompting
- Diseñar prompts efectivos sin ejemplos previos
- Aplicar técnicas de instrucción clara y contexto
- Evaluar la efectividad de diferentes enfoques zero-shot

## ¿Qué es Zero-Shot Prompting?

Zero-shot prompting es la técnica donde pedimos al modelo realizar una tarea **sin proporcionar ejemplos específicos** de cómo debe hacerla. El modelo debe basarse únicamente en:
- Su entrenamiento previo
- Las instrucciones que le damos
- El contexto que proporcionamos

### Ventajas:
- **Simplicidad**: No requiere crear ejemplos
- **Flexibilidad**: Fácil de adaptar a nuevas tareas
- **Eficiencia**: Menos tokens utilizados
- **Generalización**: Funciona para tareas diversas

### Limitaciones:
- **Inconsistencia**: Resultados pueden variar más
- **Formato**: Menos control sobre la estructura de salida
- **Tareas complejas**: Puede requerir instrucciones muy detalladas
- **Dominio específico**: Menos efectivo en nichos especializados

In [ ]:
# Configuración inicial
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
import time

# Configurar el modelo
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("GITHUB_TOKEN"),
    model="gpt-4o",
    temperature=0.7  # Balance entre creatividad y consistencia
)

print("✓ Modelo configurado para zero-shot prompting")

## Anatomía de un Prompt Zero-Shot Efectivo

Un prompt zero-shot efectivo tiene estos componentes:

1. **Contexto/Rol**: Quién debe "ser" el modelo
2. **Tarea**: Qué debe hacer específicamente
3. **Formato**: Cómo debe estructurar la respuesta
4. **Restricciones**: Qué debe evitar o considerar
5. **Input**: Los datos específicos a procesar

In [ ]:
# Ejemplo básico vs mejorado
def comparar_prompts_basico():
    texto = "El nuevo iPhone tiene una batería increíble y la cámara es fantástica, pero el precio es muy alto."
    
    # Prompt básico (menos efectivo)
    prompt_basico = f"Analiza este texto: {texto}"
    
    # Prompt mejorado (más efectivo)
    prompt_mejorado = f"""Eres un analista de sentimientos experto. 
    
Tarea: Analiza el sentimiento del siguiente texto sobre un producto.
    
Formato de respuesta:
- Sentimiento general: [Positivo/Negativo/Neutral]
- Aspectos positivos: [lista]
- Aspectos negativos: [lista]
- Puntuación de confianza: [0-10]
    
Texto a analizar: "{texto}"""
    
    print("=== COMPARACIÓN DE PROMPTS ===")
    
    # Probar prompt básico
    print("\n1. PROMPT BÁSICO:")
    print(f"Prompt: {prompt_basico}")
    try:
        response1 = llm.invoke([HumanMessage(content=prompt_basico)])
        print(f"Respuesta: {response1.content}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n" + "-"*60)
    
    # Probar prompt mejorado
    print("\n2. PROMPT MEJORADO:")
    print(f"Prompt: {prompt_mejorado[:100]}...")
    try:
        response2 = llm.invoke([HumanMessage(content=prompt_mejorado)])
        print(f"Respuesta: {response2.content}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== ANÁLISIS ===")
    print("• Prompt básico: Ambiguo, respuesta poco estructurada")
    print("• Prompt mejorado: Específico, respuesta estructurada y útil")

# Ejecutar comparación
comparar_prompts_basico()

## Técnicas de Zero-Shot por Tipo de Tarea

Diferentes tipos de tareas requieren enfoques específicos en zero-shot prompting.

In [ ]:
# 1. CLASIFICACIÓN
def zero_shot_clasificacion():
    print("=== ZERO-SHOT PARA CLASIFICACIÓN ===")
    
    emails = [
        "Reunión de equipo mañana a las 10 AM en sala de conferencias",
        "¡Felicidades! Has ganado $1,000,000! Haz clic aquí ahora",
        "Recordatorio: Tu factura de internet vence en 3 días"
    ]
    
    # Prompt optimizado para clasificación
    clasificacion_prompt = """Eres un clasificador de emails empresariales.
    
Clasifica cada email en una de estas categorías:
- TRABAJO: Relacionado con tareas laborales
- SPAM: Contenido no solicitado o sospechoso
- PERSONAL: Asuntos personales o administrativos

Responde solo con el nombre de la categoría.

Email: "{}"
Categoría:"""
    
    for i, email in enumerate(emails, 1):
        prompt = clasificacion_prompt.format(email)
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            print(f"{i}. Email: {email[:50]}...")
            print(f"   Clasificación: {response.content.strip()}\n")
        except Exception as e:
            print(f"Error en email {i}: {e}")

# Ejecutar clasificación
zero_shot_clasificacion()

In [ ]:
# 2. GENERACIÓN DE CONTENIDO
def zero_shot_generacion():
    print("=== ZERO-SHOT PARA GENERACIÓN ===")
    
    # Prompt estructurado para generación
    generacion_prompt = """Eres un copywriter creativo especializado en redes sociales.
    
Tarea: Crear un post para LinkedIn sobre el tema dado.
    
Requisitos:
- Tono profesional pero accesible
- Máximo 150 palabras
- Incluir 1-2 hashtags relevantes
- Hacer una pregunta para fomentar engagement
    
Tema: "Importancia del aprendizaje continuo en tecnología"
    
Post para LinkedIn:"""
    
    try:
        response = llm.invoke([HumanMessage(content=generacion_prompt)])
        print("Post generado:")
        print("-" * 40)
        print(response.content)
        print("-" * 40)
        
        # Análisis del resultado
        words = len(response.content.split())
        hashtags = response.content.count('#')
        questions = response.content.count('?')
        
        print(f"\nAnálisis:")
        print(f"• Palabras: {words} (límite: 150)")
        print(f"• Hashtags: {hashtags}")
        print(f"• Preguntas: {questions}")
        print(f"• Cumple requisitos: {'✓' if words <= 150 and hashtags >= 1 and questions >= 1 else '✗'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar generación
zero_shot_generacion()

In [ ]:
# 3. EXTRACCIÓN DE INFORMACIÓN
def zero_shot_extraccion():
    print("=== ZERO-SHOT PARA EXTRACCIÓN ===")
    
    # Texto de ejemplo
    texto_noticia = """Apple Inc. anunció hoy que su CEO Tim Cook se reunirá con inversores 
    el próximo 15 de marzo de 2024 en Cupertino, California. La reunión abordará los 
    resultados financieros del Q1 2024, donde la compañía reportó ingresos de $89.5 
    billones. Cook también discutirá la nueva estrategia de IA de Apple y el lanzamiento 
    del iPhone 16 programado para septiembre. Las acciones de Apple (AAPL) subieron 
    3.2% tras el anuncio."""
    
    # Prompt para extracción estructurada
    extraccion_prompt = f"""Eres un analista financiero experto en extraer datos clave.
    
Extrae la siguiente información del texto dado:
    
Formato de respuesta (JSON):
{{
  "empresa": "nombre de la empresa",
  "ceo": "nombre del CEO",
  "fecha_evento": "fecha del evento",
  "ubicacion": "ubicación del evento",
  "ingresos": "cifra de ingresos mencionada",
  "productos": ["lista de productos mencionados"],
  "cambio_acciones": "porcentaje de cambio en acciones"
}}
    
Texto: "{texto_noticia}"
    
JSON:"""
    
    try:
        response = llm.invoke([HumanMessage(content=extraccion_prompt)])
        print("Información extraída:")
        print(response.content)
        
        # Intentar parsear como JSON para validar formato
        import json
        try:
            data = json.loads(response.content)
            print("\n✓ Formato JSON válido")
            print(f"✓ Campos extraídos: {len(data)} elementos")
        except:
            print("\n✗ Formato JSON inválido - necesita refinamiento del prompt")
            
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar extracción
zero_shot_extraccion()

## Optimización de Prompts Zero-Shot

Técnicas para mejorar la efectividad de prompts zero-shot.

In [ ]:
# Técnica 1: Prompts con Roles Específicos
def roles_especificos():
    print("=== OPTIMIZACIÓN: ROLES ESPECÍFICOS ===")
    
    pregunta = "¿Cómo puedo mejorar el rendimiento de mi aplicación web?"
    
    roles = {
        "Genérico": "Responde la siguiente pregunta:",
        "Desarrollador Senior": "Eres un desarrollador senior con 10 años de experiencia en optimización web. Responde como un experto:",
        "Consultor de Performance": "Eres un consultor especializado en performance web que ha optimizado cientos de aplicaciones. Proporciona una respuesta detallada y técnica:",
        "CTO": "Eres un CTO de una startup tecnológica. Responde con una perspectiva estratégica y técnica balanceada:"
    }
    
    for rol, contexto in roles.items():
        prompt = f"{contexto}\n\n{pregunta}"
        
        print(f"\n{rol.upper()}:")
        print("-" * 30)
        
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            # Mostrar solo las primeras líneas para comparación
            preview = response.content[:200] + "..." if len(response.content) > 200 else response.content
            print(preview)
        except Exception as e:
            print(f"Error: {e}")
    
    print("\n=== OBSERVACIONES ===")
    print("• Roles específicos generan respuestas más enfocadas")
    print("• La experticia del rol influye en la profundidad técnica")
    print("• El contexto profesional afecta el tipo de soluciones propuestas")

# Ejecutar ejemplo de roles
roles_especificos()

In [ ]:
# Técnica 2: Instrucciones Graduales
def instrucciones_graduales():
    print("=== OPTIMIZACIÓN: INSTRUCCIONES GRADUALES ===")
    
    # Problema complejo para demostrar
    problema = "Necesito crear una estrategia de marketing digital para lanzar una nueva app de fitness"
    
    # Prompt con instrucciones graduales
    prompt_gradual = f"""Eres un estratega de marketing digital senior.
    
Problema: {problema}
    
Desarrolla una estrategia siguiendo este proceso paso a paso:
    
PASO 1: Análisis inicial
- Identifica el público objetivo
- Define 3 puntos de valor únicos de la app
    
PASO 2: Canales de marketing
- Selecciona 4-5 canales digitales más efectivos
- Justifica cada selección
    
PASO 3: Estrategia de contenido
- Define tipos de contenido para cada canal
- Sugiere frecuencia de publicación
    
PASO 4: Métricas y KPIs
- Define 5 métricas clave para medir éxito
- Sugiere herramientas de medición
    
Desarrolla cada paso con detalle:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_gradual)])
        print("ESTRATEGIA DESARROLLADA:")
        print("=" * 50)
        print(response.content)
        
        # Análisis de estructura
        pasos_detectados = response.content.count("PASO")
        secciones = ["análisis", "canales", "contenido", "métricas"]
        secciones_encontradas = sum(1 for seccion in secciones if seccion.lower() in response.content.lower())
        
        print("\n=== ANÁLISIS DE ESTRUCTURA ===")
        print(f"• Pasos identificados: {pasos_detectados}/4")
        print(f"• Secciones cubiertas: {secciones_encontradas}/4")
        print(f"• Estructura seguida: {'✓' if pasos_detectados >= 3 else '✗'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar instrucciones graduales
instrucciones_graduales()

## Casos de Uso Avanzados

Aplicaciones más sofisticadas de zero-shot prompting.

In [ ]:
# Caso de uso: Análisis de código
def analisis_codigo_zero_shot():
    print("=== CASO DE USO: ANÁLISIS DE CÓDIGO ===")
    
    codigo_ejemplo = '''def process_user_data(users):
    result = []
    for user in users:
        if user["age"] > 18:
            result.append({
                "name": user["name"].upper(),
                "email": user["email"],
                "status": "adult"
            })
    return result'''
    
    prompt_analisis = f"""Eres un senior code reviewer con experiencia en Python.
    
Analiza el siguiente código y proporciona:
    
1. FUNCIONALIDAD:
   - ¿Qué hace este código?
   - ¿Cuál es su propósito?
    
2. PROBLEMAS POTENCIALES:
   - Errores de lógica
   - Vulnerabilidades de seguridad
   - Problemas de rendimiento
    
3. MEJORAS SUGERIDAS:
   - Optimizaciones de código
   - Mejores prácticas
   - Manejo de errores
    
4. PUNTUACIÓN DE CALIDAD: [1-10]
    
Código a analizar:
```python
{codigo_ejemplo}
```
    
Análisis:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_analisis)])
        print("ANÁLISIS DE CÓDIGO:")
        print("=" * 50)
        print(response.content)
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar análisis de código
analisis_codigo_zero_shot()

In [ ]:
# Caso de uso: Generación de tests
def generacion_tests_zero_shot():
    print("=== CASO DE USO: GENERACIÓN DE TESTS ===")
    
    funcion_a_testear = '''def calculate_discount(price, discount_percent, user_type):
    """Calculate final price after discount.
    
    Args:
        price (float): Original price
        discount_percent (float): Discount percentage (0-100)
        user_type (str): 'premium' or 'regular'
        
    Returns:
        float: Final price after discount
    """
    if user_type == 'premium':
        discount_percent += 5  # Extra 5% for premium users
    
    discount_amount = price * (discount_percent / 100)
    return price - discount_amount'''
    
    prompt_tests = f"""Eres un QA engineer experto en testing de Python.
    
Genera tests unitarios completos para la siguiente función usando pytest.
    
Incluye tests para:
1. Casos normales (diferentes tipos de usuario y descuentos)
2. Casos edge (descuento 0%, 100%, valores negativos)
3. Casos de error (tipos incorrectos, valores inválidos)
4. Tests parametrizados cuando sea apropiado
    
Función a testear:
```python
{funcion_a_testear}
```
    
Genera código de test completo y ejecutable:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_tests)])
        print("TESTS GENERADOS:")
        print("=" * 50)
        print(response.content)
        
        # Análisis básico de los tests generados
        test_methods = response.content.count("def test_")
        assertions = response.content.count("assert")
        parametrize = response.content.count("@pytest.mark.parametrize")
        
        print("\n=== ANÁLISIS DE TESTS ===")
        print(f"• Métodos de test: {test_methods}")
        print(f"• Assertions: {assertions}")
        print(f"• Tests parametrizados: {parametrize}")
        print(f"• Cobertura estimada: {'Alta' if test_methods >= 5 else 'Media' if test_methods >= 3 else 'Baja'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar generación de tests
generacion_tests_zero_shot()

## Mejores Prácticas y Limitaciones

### ✅ Cuándo Usar Zero-Shot:
- Tareas comunes y bien definidas
- Cuando no tienes ejemplos disponibles
- Para prototipado rápido
- Tareas que requieren creatividad

### ❌ Cuándo NO Usar Zero-Shot:
- Formatos de salida muy específicos
- Tareas de dominio muy especializado
- Cuando la consistencia es crítica
- Tareas complejas que requieren patrones específicos

### 🎯 Mejores Prácticas:
1. **Sé específico**: Detalla exactamente qué quieres
2. **Define el formato**: Especifica cómo debe verse la salida
3. **Establece restricciones**: Qué debe evitar o considerar
4. **Usa roles**: Define quién debe "ser" el modelo
5. **Itera y refina**: Mejora basándote en resultados

In [ ]:
# Ejercicio final: Diseña tu propio prompt zero-shot
def ejercicio_personalizado():
    print("=== EJERCICIO PERSONAL ===")
    print("\nDiseña un prompt zero-shot para esta tarea:")
    print("'Crear una descripción de trabajo atractiva para un puesto de desarrollador Python junior'")
    print("\nEstructura sugerida:")
    print("1. Rol/Contexto del modelo")
    print("2. Tarea específica")
    print("3. Requisitos del formato")
    print("4. Restricciones importantes")
    print("\nTu prompt aquí:")
    
    # Espacio para que el estudiante diseñe su prompt
    prompt_estudiante = """# COLOCA TU PROMPT AQUÍ
    
# Ejemplo de estructura:
# Eres un [ROL]...
# Tarea: [TAREA ESPECÍFICA]...
# Formato: [ESTRUCTURA DE SALIDA]...
# Restricciones: [LIMITACIONES]...
"""
    
    print(prompt_estudiante)
    
    # Prompt de ejemplo bien estructurado
    ejemplo_prompt = """Eres un HR Business Partner senior especializado en tecnología con 8 años de experiencia reclutando desarrolladores.
    
Tarea: Redacta una descripción de trabajo atractiva y realista para un puesto de Desarrollador Python Junior.
    
Formato requerido:
- Título del puesto
- Resumen ejecutivo (2-3 líneas)
- Responsabilidades principales (4-5 puntos)
- Requisitos técnicos (separar esenciales de deseables)
- Beneficios y cultura de empresa
- Llamada a la acción
    
Restricciones:
- Máximo 400 palabras
- Lenguaje inclusivo y accesible
- Enfócate en crecimiento profesional
- Evita jerga técnica excesiva
- Sé honesto sobre el nivel junior
    
Descripción de trabajo:"""
    
    print("\n=== EJEMPLO DE PROMPT BIEN ESTRUCTURADO ===")
    print(ejemplo_prompt)
    
    # Opcional: probar el prompt ejemplo
    print("\n¿Quieres ver el resultado del prompt ejemplo? (y/n)")
    # En un entorno real, aquí habría input del usuario
    
ejercicio_personalizado()

## Conceptos Clave Aprendidos

1. **Zero-shot prompting** es efectivo para tareas comunes sin necesidad de ejemplos
2. **Estructura clara** del prompt mejora significativamente los resultados
3. **Roles específicos** generan respuestas más enfocadas y expertas
4. **Instrucciones graduales** ayudan con tareas complejas
5. **Formato definido** asegura salidas consistentes y útiles

## Próximos Pasos

En el siguiente notebook exploraremos **Few-Shot Prompting**, donde aprenderemos a usar ejemplos específicos para guiar el comportamiento del modelo y lograr mayor consistencia en tareas especializadas.

### Para Practicar:
1. Diseña prompts zero-shot para diferentes tipos de tareas
2. Experimenta con diferentes roles y contextos
3. Compara resultados con y sin estructura específica
4. Mide la consistencia en múltiples ejecuciones